In [1]:
import os
import pandas as pd
import os, sys, json
from datetime import date
import parsl
from parsl.app.app import python_app
import subprocess
import argparse
import h5py

In [5]:
whereis_script = '/lus/grand/projects/covid-ct/imlab/users/temi/projects/TFXcan/TFpred_pipeline/' #os.path.dirname(sys.argv[0]) # or os.path.dirname(__file__)
os.chdir(whereis_script), os.getcwd()

(None,
 '/lus/grand/projects/covid-ct/imlab/users/temi/projects/TFXcan/TFpred_pipeline')

In [6]:
project_dir = f'/lus/grand/projects/covid-ct/imlab/users/temi/projects/TFXcan/TFpred_pipeline'
predictions_dir = f'{project_dir}/prediction_folder/enformer_predictions/'

In [7]:

mtdata = './aggregation_config_freedman_FOXA1.json'

agg_type = 'aggByMean'
TF = 'FOXA1'
dataset = 'cistrome'
data_date = '2023-01-11'

script_path = f'{project_dir}/pipelines'
utils_path = os.path.join(script_path, 'utilities')
sys.path.append(utils_path)

In [10]:
# one h5 file
# chr12_57355008_57355017_predictions.h5
# chr8_133158323_133158332_predictions.h5
# chr2_201510974_201510983_predictions.h5
# chr11_133631801_133631810_predictions.h5
one_file = f'./prediction_folder/enformer_predictions/cistrome_FOXA1/predictions_2023-01-11/cistrome/chr12_57355008_57355017_predictions.h5'
f = h5py.File(one_file, 'r')
list(f.keys())

['chr12_57355008_57355017']

In [15]:
test_predictions = ['chr12_57355008_57355017', 'chr8_133158323_133158332', 'chr2_201510974_201510983', 'chr11_133631801_133631810']

In [6]:
dset = f['chr12_57355008_57355017']

In [8]:
dset.shape

(17, 5313)

In [14]:
h5_db = f'{whereis_script}/prediction_folder/predictions_db'
os.makedirs(h5_db)

In [16]:
h5files = [f'./prediction_folder/enformer_predictions/cistrome_FOXA1/predictions_2023-01-11/cistrome/{i}_predictions.h5' for i in test_predictions]
h5files

['./prediction_folder/enformer_predictions/cistrome_FOXA1/predictions_2023-01-11/cistrome/chr12_57355008_57355017_predictions.h5',
 './prediction_folder/enformer_predictions/cistrome_FOXA1/predictions_2023-01-11/cistrome/chr8_133158323_133158332_predictions.h5',
 './prediction_folder/enformer_predictions/cistrome_FOXA1/predictions_2023-01-11/cistrome/chr2_201510974_201510983_predictions.h5',
 './prediction_folder/enformer_predictions/cistrome_FOXA1/predictions_2023-01-11/cistrome/chr11_133631801_133631810_predictions.h5']

In [ ]:
motifs_list = log_data.loc[log_data['sequence_type'] == pred_type, ].motif.values.tolist()

In [39]:
log_data = pd.read_csv(f'{project_dir}/prediction_folder/predictions_log/{dataset}_{TF}/predictions_log_{data_date}.csv')
log_data.iloc[0:5, ], log_data.shape

(                      motif individual     status sequence_type
 0   chr18_60106170_60106179   cistrome  completed           ref
 1     hr1_23967233_23967242   cistrome  completed           ref
 2  chr3_109912469_109912478   cistrome  completed           ref
 3   chr16_58215584_58215593   cistrome  completed           ref
 4   chr10_59714479_59714488   cistrome  completed           ref,
 (46888, 4))

In [40]:
# compress data into one hdf5 file
# you alse need to save the name of the files/motifs
motifs_list = list(set(log_data.motif.values.tolist()))
len(motifs_list) 


46888

In [41]:
h5files = [f'./prediction_folder/enformer_predictions/cistrome_FOXA1/predictions_2023-01-11/cistrome/{i}_predictions.h5' for i in motifs_list]
h5files[0:5]

['./prediction_folder/enformer_predictions/cistrome_FOXA1/predictions_2023-01-11/cistrome/chr4_45874952_45874961_predictions.h5',
 './prediction_folder/enformer_predictions/cistrome_FOXA1/predictions_2023-01-11/cistrome/chr1_44660554_44660563_predictions.h5',
 './prediction_folder/enformer_predictions/cistrome_FOXA1/predictions_2023-01-11/cistrome/chr4_150900878_150900887_predictions.h5',
 './prediction_folder/enformer_predictions/cistrome_FOXA1/predictions_2023-01-11/cistrome/chr3_137061588_137061597_predictions.h5',
 './prediction_folder/enformer_predictions/cistrome_FOXA1/predictions_2023-01-11/cistrome/chr10_56496697_56496706_predictions.h5']

In [42]:
# first filter if the file is present in the folder
h5files = [dt for dt in h5files if os.path.isfile(dt) ]
len(h5files)

46887

In [45]:
pd.DataFrame(motifs_list, columns=['motif']).to_csv()

,motif
0,chr4_45874952_45874961
1,chr1_44660554_44660563
2,chr4_150900878_150900887
3,chr3_137061588_137061597
4,chr10_56496697_56496706
...,...
46883,chr21_42801693_42801702
46884,chr14_102154568_102154577
46885,chr4_155339892_155339901
46886,chr1_220612585_220612594


In [ ]:
def 

In [46]:
with h5py.File(f"{h5_db}/{dataset}_{data_date}.hdf5", "w") as f_dst:
    #h5files = [f for f in os.listdir(f'{project_dir}/') if f.endswith(".h5")]

    dset = f_dst.create_dataset(f'{dataset}_dataset', shape=(len(h5files), 17, 5313), dtype='f4')

    for i, filename in enumerate(motifs_list):
        #print(filename)
        with h5py.File(h5files[i]) as f_src:
            dset[i] = f_src[filename]

pd.DataFrame(motifs_list, columns=['motif']).to_csv(f"{h5_db}/{dataset}_{data_date}.csv")

KeyboardInterrupt: 

In [47]:
#output_predictions = {}

with h5py.File(f"{h5_db}/{dataset}_{data_date}.hdf5", "r") as f_dst:
    #h5files = [f for f in os.listdir(f'{project_dir}/') if f.endswith(".h5")]
    output_predictions = f_dst.get(f'{dataset}_dataset')[()] 
    motifs = list(f_dst.keys())

In [48]:
output_predictions.shape

(46887, 17, 5313)

In [27]:
motifs

['mydataset']

In [34]:
print(f'[INFO] Currently on {dataset_type}')

save_dir = f'{base_path}/aggregated_predictions/{dataset_type}_{TF}_{todays_date}'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

if individuals is None:
    ids_names = [dataset_type]
elif isinstance(individuals, list):
    ids_names = individuals

log_data_all = pd.read_csv(log_file)

# use parsl if the num of rows of log_data is more than 10000
use_parsl = False

predict_utils_one = f'{script_path}/utilities/utility_functions.py'
exec(open(predict_utils_one).read(), globals(), globals())

if use_parsl == True:
    #bpath = os.path.join(base_path, 'modeling_pipeline')
    print(f'[INFO] Using parsl. Aggregation will be split into multiple files, and batches will be collected into one later.')
    parsl_params = {'working_dir':base_path, 'job_name':'aggregate_predictions', 'queue':"preemptable", 'walltime':"05:59:00", 'num_of_full_nodes':1, 'min_num_blocks':0, 'max_num_blocks':10}
    parsl.load(parslConfiguration.localParslConfig_htpool(parsl_params))

collection_fxn = return_prediction_function(use_parsl)

[INFO] Currently on freedman


In [27]:
log_data_all = pd.read_csv(log_file)
log_data_all.drop_duplicates(subset=['motif'])

,motif,individual,status,sequence_type
0,chr12_92688598_92688607,LuCaP_136,completed,var
1,hr9_122819978_122819987,LuCaP_136,completed,var
2,chr9_18363310_18363319,LuCaP_136,completed,var
3,chr5_160240472_160240481,LuCaP_136,completed,var
4,chr20_11244716_11244725,LuCaP_136,completed,var
...,...,...,...,...
9099,chrX_68854754_68854763,LuCaP_141,completed,var
9100,chr9_122819978_122819987,LuCaP_141,completed,var
9101,chr7_3576376_3576385,LuCaP_141,completed,var
9174,chr1_176593308_176593317,LuCaP_141,completed,var


In [32]:
log_data_all.shape

(9103, 4)

In [36]:
ids_names = ['LuCaP_141']

for each_id in ids_names:
    log_data = log_data_all.loc[log_data_all['individual'] == each_id, ]
    log_data = log_data.drop_duplicates(subset=['motif']) #.iloc[1:1000, ]

    print(log_data)

    print(log_data.shape)

    range_batches = generate_batch(range(0, log_data.shape[0]), batch_n=8)

    ind_path = os.path.join(enformer_predictions_path, each_id)

    app_futures = []
    for i, range_batch in enumerate(range_batches):

        print(list(range_batches))
        app_futures.append(f'{i}: {len(range_batch)}')

        # print(i)
        # app_futures.append(collection_fxn(each_id=each_id, log_data=log_data.iloc[range_batch, ], predictions_path=ind_path, TF=TF, agg_types=[agg_type], base_path=base_path, save_dir=save_dir, batch_num=i))

    print(app_futures)

    if use_parsl == True:
        app_execs = [r.result() for r in app_futures]

    # cmd = f"for i in `ls {save_dir}/{each_id}_{agg_type}_{TF}_batch_*.csv.gz`; do zcat $i | sed '1d'; done | pigz -c >{save_dir}/{each_id}_{agg_type}_{TF}.csv.gz"    
    # p = subprocess.Popen(cmd, shell=True)
    # print(f'[INFO] Status: Aggregation complete for {log_data.shape[0]} predictions for {each_id}.')

                          motif individual     status sequence_type
9098    chr12_92688598_92688607  LuCaP_141  completed           var
9099     chrX_68854754_68854763  LuCaP_141  completed           var
9100   chr9_122819978_122819987  LuCaP_141  completed           var
9101       chr7_3576376_3576385  LuCaP_141  completed           var
9102     chr3_24951897_24951906  LuCaP_141  completed           var
...                         ...        ...        ...           ...
18195   chr12_23533600_23533609  LuCaP_141  completed           var
18196  chr5_102611585_102611594  LuCaP_141  completed           var
18197    chr8_84108965_84108974  LuCaP_141  completed           var
18198   chr21_24828757_24828766  LuCaP_141  completed           var
18199    chr4_79041264_79041273  LuCaP_141  completed           var

[9102 rows x 4 columns]
(9102, 4)
[range(1138, 2276), range(2276, 3414), range(3414, 4552), range(4552, 5690), range(5690, 6828), range(6828, 7966), range(7966, 9102)]
['0: 1138']
